In [5]:
!pip install -U sagemaker

     |████████████████████████████████| 406 kB 21.8 MB/s eta 0:00:01
  Using cached smdebug_rulesconfig-1.0.1-py2.py3-none-any.whl (20 kB)
  Created wheel for sagemaker: filename=sagemaker-2.35.0-py2.py3-none-any.whl size=571601 sha256=366ea3afbbb7c77e338785ff2b5b3abe16f7b380bb753fe092e3c786b018b0c3
  Stored in directory: /home/ec2-user/.cache/pip/wheels/db/41/e3/d3214bcdfad852b785fcb37e5db15d64e5fa1c1ac33ca4eb1b
Successfully built sagemaker
  Attempting uninstall: smdebug-rulesconfig
    Found existing installation: smdebug-rulesconfig 1.0.0
    Uninstalling smdebug-rulesconfig-1.0.0:
      Successfully uninstalled smdebug-rulesconfig-1.0.0
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.19.0
    Uninstalling sagemaker-2.19.0:
      Successfully uninstalled sagemaker-2.19.0
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/bin/python -m pip install --upgrade pip' command.


In [127]:
import os
import sagemaker
import numpy as np
from sagemaker.pytorch import PyTorch

ON_SAGEMAKER_NOTEBOOK = True

sagemaker_session = sagemaker.Session()
if ON_SAGEMAKER_NOTEBOOK:
    role = sagemaker.get_execution_role()
else:
    role = "[YOUR ROLE]"

import boto3
client = boto3.client('sagemaker-runtime')

In [128]:
bucket = "feidemo"
key = "interdev/junction-data"   
training_data_uri="s3://{}/{}".format(bucket, key)

In [129]:
estimator = PyTorch(entry_point='ptModelCode.py',
                    role=role,
                    framework_version='1.8',
                    instance_count=1,
                    instance_type='ml.p2.xlarge',
                    py_version='py3',
                    # available hyperparameters: emsize, nhid, nlayers, lr, clip, epochs, batch_size,
                    #                            bptt, dropout, tied, seed, log_interval
                    )

In [130]:
estimator.fit(training_data_uri)

2021-04-16 21:30:45 Starting - Starting the training job...
2021-04-16 21:30:57 Starting - Launching requested ML instancesProfilerReport-1618608644: InProgress
......
2021-04-16 21:32:14 Starting - Preparing the instances for training............
2021-04-16 21:33:59 Downloading - Downloading input data...
2021-04-16 21:34:40 Training - Downloading the training image.....................
2021-04-16 21:38:09 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-04-16 21:38:09,906 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-04-16 21:38:09,931 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-04-16 21:38:09,939 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-04-16 21:38:10,437 sagemaker-training-toolkit INFO     Invoking user script


In [2]:
pytorch_estimator=PyTorch.attach(training_job_name='pytorch-training-2021-04-14-21-34-36-987')


2021-04-14 21:50:04 Starting - Preparing the instances for training
2021-04-14 21:50:04 Downloading - Downloading input data
2021-04-14 21:50:04 Training - Training image download completed. Training in progress.
2021-04-14 21:50:04 Uploading - Uploading generated training model
2021-04-14 21:50:04 Completed - Training job completed


In [132]:
estimator.model_data

's3://sagemaker-us-east-1-973994155064/pytorch-training-2021-04-16-21-30-44-860/output/model.tar.gz'

In [133]:
predictor=estimator.deploy(initial_instance_count=1, instance_type='ml.m5.4xlarge', entry_point='ptInfCode.py')

---------------!

In [32]:
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

transformations = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [134]:
with open("/home/ec2-user/SageMaker/manyFrameworks/Test/Test/Priority/P1.png", 'rb') as f:
    image_bytes=f.read()
    predictor.predict(image_bytes)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (0) from model with message "Your invocation timed out while waiting for a response from container model. Review the latency metrics for each container in Amazon CloudWatch, resolve the issue, and try again.". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/pytorch-training-2021-04-16-22-04-59-217 in account 973994155064 for more information.

In [16]:
image_set = ImageFolder("/home/ec2-user/SageMaker/manyFrameworks/Test/Test/", transform=transformations)
data_loader=DataLoader(image_set, batch_size=4, shuffle=True, num_workers=1)

In [136]:
from sagemaker.pytorch import PyTorchModel
pytorch_model = PyTorchModel(model_data='s3://sagemaker-us-east-1-973994155064/pytorch-training-2021-04-16-21-30-44-860/output/model.tar.gz', role=role, entry_point='ptInfCode.py', framework_version='1.7',py_version='py3')
predictor = pytorch_model.deploy(instance_type='ml.m5.4xlarge', initial_instance_count=1)

---------------!

In [149]:
im_name="/home/ec2-user/SageMaker/manyFrameworks/Test/Test/Signal/S2.png"

response = client.invoke_endpoint(
EndpointName='pytorch-inference-2021-04-16-22-19-41-647',
ContentType='application/x-image',
Body=open(im_name, 'rb').read())

In [150]:
import json
json.loads(response['Body'].read().decode("utf-8"))

[{'prediction': 'Signal', 'score': '56100.994873046875%'}]

''

In [38]:

from torchvision import models
import torch.nn as nn
import torch


n_classes=3

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.resnet34(pretrained=True)

n_features = model.fc.in_features
model.fc = nn.Linear(n_features, n_classes)

with open('model.pth', 'rb') as f:
    model.load_state_dict(torch.load(f,map_location=torch.device('cpu')))

model.to(device).eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [110]:
def input_fn(image_bytes):
#     logger.info('Deserializing the input data.')
    image_data=image_bytes
    
    image_transform = transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    return image_transform(image_data)

#     raise Exception(f'Requested unsupported ContentType in content_type: {content_type}')

def predict_fn(input_data, model):
#     logger.info('Generating prediction based on input parameters.')
#     if torch.cuda.is_available():
#         input_data = input_data.view(1, 3, 224, 224).cuda()
#     else:
    input_data = input_data.view(1, 3, 224, 224)
    with torch.no_grad():
        model.eval()
        out = model(input_data)
        
        _, preds = torch.max(out, 1)
    return ps

def output_fn(prediction_output, accept='application/json'):
#     logger.info('Serializing the generated output.')
    classes = {0: 'Priority', 1: 'Roundabout', 2: 'Signal'}

    topk, topclass = prediction_output.topk(3, dim=1)
    result = []

    for i in range(3):
        pred = {'prediction': classes[topclass.cpu().numpy()[0][i]], 'score': f'{topk.cpu().numpy()[0][i] * 100}%'}
        print(f'Adding pediction: {pred}')
        result.append(pred)
    print(result)

#     if accept == 'application/json':
#         return json.dumps(result), accept
#     raise Exception(f'Requested unsupported ContentType in Accept: {accept}')



In [111]:
from PIL import Image
# image_bytes=Image.open("/home/ec2-user/SageMaker/manyFrameworks/Test/Test/Priority/P1.png")
# image_bytes=image_bytes.convert('RGB')
# # image_bytes=image_bytes.view(1,3,224,224)



with open("/home/ec2-user/SageMaker/manyFrameworks/Test/Test/Roundabout/R1.png", 'rb') as f:
    img=f.read()
#     image_bytes=Image.open("/home/ec2-user/SageMaker/manyFrameworks/Test/Test/Priority/P1.png")
    
    image_bytes=Image.open(io.BytesIO(img))
    image_bytes=image_bytes.convert('RGB')
    image_data=input_fn(image_bytes)
    ps=predict_fn(image_data, model)
ps

tensor([[8.3474, 0.6464, 1.7348]])

In [112]:
x=output_fn(ps, accept='application/json')

Adding pediction: {'prediction': 'Priority', 'score': '834.7400665283203%'}
Adding pediction: {'prediction': 'Signal', 'score': '173.47679138183594%'}
Adding pediction: {'prediction': 'Roundabout', 'score': '64.63744044303894%'}
[{'prediction': 'Priority', 'score': '834.7400665283203%'}, {'prediction': 'Signal', 'score': '173.47679138183594%'}, {'prediction': 'Roundabout', 'score': '64.63744044303894%'}]


In [1]:
from sagemaker.pytorch.model import PyTorchPredictor

In [3]:
predictorPT=PyTorchPredictor('pytorch-inference-2021-04-16-02-19-22-715',sagemaker.Session())

In [30]:
from PIL import Image
import io


with open("/home/ec2-user/SageMaker/manyFrameworks/Test/Test/Priority/P1.png", 'rb') as f:
    img=f.read()
#     image_bytes=Image.open("/home/ec2-user/SageMaker/manyFrameworks/Test/Test/Priority/P1.png")
    
    image_bytes=Image.open(io.BytesIO(img))
    image_bytes=image_bytes.convert('RGB')
    predictor.predict(image_bytes)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from model with message "'NoneType' object has no attribute 'view'
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/sagemaker_inference/transformer.py", line 126, in transform
    result = self._transform_fn(self._model, input_data, content_type, accept)
  File "/opt/conda/lib/python3.6/site-packages/sagemaker_inference/transformer.py", line 216, in _default_transform_fn
    prediction = self._predict_fn(data, model)
  File "/opt/ml/model/code/ptInfCode.py", line 62, in predict_fn
    input_data = input_data.view(1, 3, 224, 224)
AttributeError: 'NoneType' object has no attribute 'view'
". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/pytorch-inference-2021-04-16-19-23-37-430 in account 973994155064 for more information.